In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import tushare as ts
import numpy as np
import random as rd
import time,sys,os,sqlite3
import akshare as ak
import traceback

In [128]:
root = Path("./../..")
datapath = root.joinpath(Path("data/tushare_old/all"))
datapath.resolve()

WindowsPath('D:/gitee/data/tushare_old/all')

In [75]:
pa = pd.read_excel(datapath.joinpath(Path("000001.xlsx")).resolve())

In [69]:
pa.index = pa["date"]
pa.sort_index(inplace=True)
pa["avg5"] = pa.close.rolling(5).mean().round(3)
pa1 = pa.dropna(inplace=False)
pa1

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,avg5
date,,,,,,,,,,,,,,,,
2021-06-07,2021-06-07,24.48,24.48,24.30,23.93,447914.09,-0.24,-0.98,24.084,24.352,23.944,545619.87,613158.06,545848.34,0.23,24.084
2021-06-08,2021-06-08,24.15,24.57,24.53,23.95,570077.94,0.23,0.95,24.206,24.345,23.994,534631.87,572928.39,549491.07,0.29,24.206
2021-06-09,2021-06-09,24.43,24.88,24.65,24.18,453831.53,0.12,0.49,24.358,24.309,24.049,525892.77,526385.17,545519.38,0.23,24.358
2021-06-10,2021-06-10,24.66,25.06,24.20,24.15,530215.19,-0.45,-1.83,24.444,24.250,24.106,551755.64,528997.55,544547.52,0.27,24.444
2021-06-11,2021-06-11,24.31,24.35,23.37,23.12,834665.06,-0.83,-3.43,24.210,24.137,24.108,567340.76,563223.99,558091.50,0.43,24.210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14,2022-10-14,11.45,11.63,11.53,11.40,1096061.62,0.19,1.68,11.484,11.738,12.163,777916.73,781880.89,946408.95,0.56,11.484
2022-10-17,2022-10-17,11.42,11.52,11.48,11.34,1024822.75,-0.05,-0.43,11.486,11.686,12.101,789665.24,793990.26,900085.23,0.53,11.486
2022-10-18,2022-10-18,11.55,11.64,11.48,11.46,946837.12,0.00,0.00,11.486,11.619,12.028,895981.99,809711.74,861292.59,0.49,11.486


In [143]:
def cal_ma(df,seq):
    """
    用收盘价计算n天ma，在df上直接添加相应列
    df中需要有日期列
    """
    df.sort_values(by='date', ascending=True, inplace=True)
    for i in seq:
        df["avg"+str(i)] = df.close.rolling(i).mean().round(3)
        df["v_avg"+str(i)] = df.volume.rolling(i).mean().round(2)
    res = df.dropna(inplace=False).sort_values(by='date', ascending=False, inplace=False)
    return res

def acc_check(df,seq):
    for i in seq:
        if sum(df["v_avg"+str(i)] - df["v_ma"+str(i)].astype(float) > 0.001): print("voluma ma {} 计算有问题".format(i)) 
        if sum(df["avg"+str(i)] - df["ma"+str(i)].astype(float)  > 0.001): print("voluma ma {} 计算有问题".format(i)) 
    return 


    

In [125]:
pa1 = cal_ma(pa,[3,5,10,20,60])

In [126]:
acc_check(pa1,[5,10,20])

In [111]:
pa1

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,avg3,avg5,avg10,avg20,avg60,v_avg3,v_avg5,v_avg10,v_avg20,v_avg60
0,2022-10-20,11.23,11.31,11.20,11.16,781923.19,-0.09,-0.80,11.396,11.471,11.866,954726.87,810438.66,811635.55,0.40,11.323,11.396,11.471,11.865,12.289,884250.00,954726.87,810438.66,811635.55,977280.23
1,2022-10-19,11.42,11.49,11.29,11.28,923989.69,-0.19,-1.66,11.424,11.537,11.956,968865.44,824018.14,864590.28,0.48,11.417,11.424,11.537,11.955,12.326,965216.52,968865.44,824018.14,864590.28,977605.82
2,2022-10-18,11.55,11.64,11.48,11.46,946837.12,0.00,0.00,11.486,11.619,12.028,895981.99,809711.74,861292.59,0.49,11.497,11.486,11.619,12.028,12.361,1022573.83,895981.99,809711.74,861292.59,974799.28
3,2022-10-17,11.42,11.52,11.48,11.34,1024822.75,-0.05,-0.43,11.486,11.686,12.101,789665.24,793990.26,900085.23,0.53,11.450,11.486,11.686,12.101,12.393,991166.79,789665.24,793990.26,900085.23,978243.76
4,2022-10-14,11.45,11.63,11.53,11.40,1096061.62,0.19,1.68,11.484,11.738,12.163,777916.73,781880.89,946408.95,0.56,11.490,11.484,11.738,12.163,12.423,836083.35,777916.73,781880.89,946408.95,987807.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2021-08-30,18.21,18.29,17.72,17.55,1252211.00,-0.67,-3.64,18.618,19.244,19.101,899937.51,976489.46,926998.98,0.65,18.190,18.618,19.244,19.100,20.827,977550.10,899937.51,976489.46,926998.98,817229.69
274,2021-08-27,18.56,18.88,18.39,18.31,728820.25,-0.07,-0.38,18.934,19.467,19.115,887865.19,913973.38,922384.56,0.38,18.670,18.934,19.467,19.115,20.941,792841.10,887865.19,913973.38,922384.56,808971.83
275,2021-08-26,19.20,19.26,18.46,18.41,951619.06,-0.70,-3.65,19.140,19.617,19.080,1065026.74,897379.87,933451.34,0.49,18.993,19.140,19.617,19.080,21.031,839552.10,1065026.74,897379.86,933451.34,803506.50
276,2021-08-25,19.42,19.47,19.16,18.95,698084.00,-0.20,-1.03,19.516,19.760,19.055,1004698.05,868863.76,929489.29,0.36,19.273,19.516,19.760,19.055,21.121,919628.88,1004698.05,868863.76,929489.29,795938.30


In [135]:
# 读取股票列表
stock_pool = pd.read_excel('D:/gitee/data/tushare_old/allstock_221020.xlsx',dtype = 'str')
stock_list = stock_pool[['code','display_name']]
stock_list.index = stock_list['code']
stock_list = stock_list['display_name']
stock_dict = stock_list.to_dict()

In [144]:
for i,j in enumerate(os.listdir(datapath)):
    temp = pd.read_excel(str(datapath.resolve())+r"/"+j,dtype = 'str')
    temp["code"] = j[:6]
    temp["name"] = stock_dict[j[:6]]
    temp = cal_ma(temp,[3,5,10,20,60])
    if not ((i+1) % 100): 
        print("完成 {}".format(i+1))
    temp = cal_ma(temp,[3,5,10,20,60])
    acc_check(temp,[5,10,20])
print(temp)


完成 1000
           date  open  high close   low    volume price_change p_change  \
0    2022-10-20  5.19  5.23  5.13   5.1   42240.7        -0.03    -0.58   
1    2022-10-19  5.29   5.3  5.16  5.16  49064.32        -0.11    -2.09   
2    2022-10-18  5.25  5.36  5.27  5.21  72369.34         0.01     0.19   
3    2022-10-17  5.17   5.3  5.26   5.1  73311.64         0.05     0.96   
4    2022-10-14  5.13  5.26  5.21  5.06   79578.8         0.13     2.56   
..          ...   ...   ...   ...   ...       ...          ...      ...   
272  2021-08-31  4.98  5.07     5   4.9  15979.41         0.04     0.81   
273  2021-08-30  4.87  5.08  4.96  4.85  29756.32         0.09     1.85   
274  2021-08-27  4.88   4.9  4.87  4.76  18151.66        -0.03    -0.61   
275  2021-08-26  4.95  4.98   4.9  4.86  20566.95        -0.05    -1.01   
276  2021-08-25   4.8  4.98  4.95  4.71   39019.4         0.17     3.56   

       ma5   ma10   ma20     v_ma5    v_ma10     v_ma20 turnover    code  \
0    5.206  5.0

In [145]:
temp

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,code,name,avg3,v_avg3,avg5,v_avg5,avg10,v_avg10,avg20,v_avg20,avg60,v_avg60
0,2022-10-20,40.01,40.81,40.06,39.66,16596.01,-0.33,-0.82,40.54,39.495,39.293,16202.76,16743.4,18456.54,689009,九号公司,40.557,14624.05,40.540,16202.76,39.495,16743.40,39.293,18456.54,45.932,27142.84
1,2022-10-19,41.76,41.76,40.39,40.14,11958.92,-0.83,-2.01,40.38,39.289,39.31,16198.89,17302.6,18872.54,689009,九号公司,40.937,15366.26,40.380,16198.89,39.289,17302.60,39.310,18872.54,46.093,27564.62
2,2022-10-18,41.03,41.86,41.22,40.54,15317.22,0.02,0.05,40.256,39.085,39.358,19104.47,17462.74,19867.76,689009,九号公司,40.750,17486.30,40.256,19104.47,39.085,17462.74,39.358,19867.76,46.188,27904.39
3,2022-10-17,39.52,41.65,41.2,39.51,18822.63,1.37,3.44,39.494,38.931,39.427,18332.36,17603.65,20021.49,689009,九号公司,40.097,17906.11,39.494,18332.36,38.931,17603.64,39.426,20021.49,46.291,28395.43
4,2022-10-14,39.84,40.66,39.83,39.25,18319.04,0.57,1.45,38.754,38.671,39.552,17521.41,17499.74,19848.98,689009,九号公司,39.620,20460.83,38.754,17521.41,38.671,17499.74,39.552,19848.97,46.413,29042.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2021-08-30,76.89,78,74.66,72.63,34943.67,-3.39,-4.34,79.124,71.983,68.394,43201.91,25827.28,19109.83,689009,九号公司,79.270,39491.02,79.124,43201.91,71.983,25827.28,68.394,19109.83,72.605,18791.08
274,2021-08-27,83.18,83.4,78.05,76.66,37110.19,-7.05,-8.28,77.392,71.03,68.026,38912.81,23273.82,17809.11,689009,九号公司,83.077,51968.47,77.392,38912.81,71.030,23273.82,68.026,17809.11,72.617,18568.37
275,2021-08-26,87.76,87.9,85.1,81.18,46419.2,-0.98,-1.14,74.394,69.605,67.543,33204.43,21128.18,16672.85,689009,九号公司,80.970,47985.23,74.394,33204.43,69.605,21128.18,67.543,16672.85,72.613,18224.29
276,2021-08-25,80,86.08,86.08,79,72376.02,14.35,20.01,70.386,67.871,66.69,24980.86,18352.45,15161.82,689009,九号公司,74.603,37011.55,70.386,24980.86,67.871,18352.45,66.690,15161.82,72.531,18030.70
